In [1]:
import sys
sys.path.append("../..")
import pandas as pd
from pyadlml.dataset.homeassistant import hass_db_2_df

In [2]:
db_url = 'sqlite:////' + '/home/chris/data/home-assistant.db'

In [13]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import pandas as pd
from sqlalchemy import create_engine, text


def hass_db_2_df(db_url, limit=5000000):
    """
    Parameters
    ----------
    db_url : String
        url to the database 
    """
    query = f"""
    SELECT entity_id, state, last_changed
    FROM states
    WHERE
        state NOT IN ('unknown', 'unavailable')
    ORDER BY last_changed DESC
    LIMIT {limit}
    """
    df = pd.read_sql_query(query, db_url)
    return df


In [16]:
df = hass_db_2_df(db_url)

In [21]:
device_list = [
    'binary_sensor.ping_chris_laptop',
    'binary_sensor.ping_chris_pc',
    'binary_sensor.ping_chris_smartphone',
    'binary_sensor.pir_motion_floor',
    'binary_sensor.pir_motion_mirror',
    'binary_sensor.pir_motion_monitor_left',
    'binary_sensor.pir_motion_monitor_right',
    'binary_sensor.us_distance_floor',
    'binary_sensor.us_distance_mirror',
    'binary_sensor.us_distance_monitor_left',
    'binary_sensor.us_distance_monitor_right',
    'switch.chris_computer',
    'light.storu_shelf_front',
    'light.storu_desk',
    'light.storu_coffee',
    'light.wall',
    'light.storu_mirror', 
    'switch.senseo_one_cup',
    'switch.printer',
    'switch.fan',
    'switch.storu_mirror_light',
    'light.mirror',
]

In [32]:
def hass_db_2_data(db_url, device_list, start_time=None):
    """
    Parameters
    ----------
    Returns
    -------
    df : pd.DataFrame
    df2 : pd.DataFrame
        
    """
    device_dict = {device_list[i]:i for i in range(len(device_list))}
    
    df = hass_db_2_df(db_url)
    df = df[df['entity_id'].isin(device_list)]
    df['time'] = pd.to_datetime(df['last_changed'])
    df['val'] = (df['state'] == 'on').astype(int)
    df['device'] = df['entity_id']
    df = df[[ 'time', 'device','val']]
    
    # encode data
    df['device'] = df["device"].replace(device_dict)
    
    # create feature map
    df2 = pd.DataFrame(device_list, columns=['devices'])
    df2 = df2.reset_index()
    df2.columns = ['id', 'devices']
    return df, df2

In [33]:
df, df2 = hass_db_2_data(db_url, device_list)

In [34]:
df2.to_csv('/home/chris/data/device_mapping.csv', sep=',', index=False)

In [35]:
df.to_csv('/home/chris/data/data.csv', sep=',', header=True, index=False)